# Columnstore Compression

Create a nonclustered columnstore index on 5 columns from dbo.ParkingTickets

In [ ]:
USE ParkingTickets
GO

CREATE COLUMNSTORE INDEX CI_ParkingTickets
ON ParkingTickets.dbo.ParkingTickets (
    Ticket_number, -- bigint
    license_state, -- char(2)
    license_type, -- char(3)
    car_make, -- char(4)
    issue_date -- datetime2(0)
);

View the row groups that were created for this columnstore index.

  

Note how most contain the maximum number of rows (1,048,576).

In [ ]:
select * 
from ParkingTickets.sys.column_store_row_groups
where OBJECT_id = OBJECT_ID('ParkingTickets.dbo.ParkingTickets')
ORDER BY row_group_id;

But what happens if we add a row to the table? Let's add one and see.

In [ ]:
INSERT INTO ParkingTickets.dbo.ParkingTickets (Ticket_number, license_state, license_type, issue_date, violation_location, 
    violation_code, badge)
VALUES (7123456, 'NY', 'PAS', '20210101', '3 Easy St.', '1337','54');


Now re-run the query of row groups.

Note how now one of the row groups is in an "OPEN" state, as a record has been added and is being held in the delta rowgroup

In [ ]:
select * 
from ParkingTickets.sys.column_store_row_groups
where OBJECT_id = OBJECT_ID('ParkingTickets.dbo.ParkingTickets')
ORDER BY row_group_id;

Let's take a quick look at all the segments in this columnstore index

In [ ]:
select seg.*
from ParkingTickets.sys.column_store_segments seg
	inner join ParkingTickets.sys.partitions as p 
		ON seg.partition_id = p.partition_id
	where p.object_id = OBJECT_ID('ParkingTickets.dbo.ParkingTickets');

In [ ]:
select DISTINCT seg.HOBT_ID 
from ParkingTickets.sys.column_store_segments seg
	inner join ParkingTickets.sys.partitions as p 
		ON seg.partition_id = p.partition_id
	where p.object_id = OBJECT_ID('ParkingTickets.dbo.ParkingTickets');

Paste hobt\_id from previous query into this one to see all dictionaries associated with the columnstore index:

In [ ]:
SELECT *
FROM ParkingTickets.sys.column_store_dictionaries
WHERE hobt_id = <hobt_id>

Get column Name/ID for each column:

In [ ]:
select
    ic.index_column_id,
    COL_NAME(ic.object_id, ic.column_id) AS column_name
from sys.objects o
inner join sys.indexes i on o.object_id = i.object_id
inner join sys.index_columns ic on ic.object_id = o.object_id
where o.name = 'ParkingTickets' and i.name = 'CI_ParkingTickets';


View all the segments for a particular rowgroup. Note the "encoding\_type" column, which indicates what type of encoding is used for that particular segment. See [documentation](https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-column-store-segments-transact-sql?view=sql-server-ver15) for full definitions.

- 1 = VALUE_BASED - non-string/binary with no dictionary (similar to 4 with some internal variations)
- 2 = VALUE_HASH_BASED - non-string/binary column with common values in dictionary
- 3 = STRING_HASH_BASED - string/binary column with common values in dictionary
- 4 = STORE_BY_VALUE_BASED - non-string/binary with no dictionary
- 5 = STRING_STORE_BY_VALUE_BASED - string/binary with no dictionary
    

In [ ]:
select *
FROM sys.column_store_segments
WHERE hobt_id = 72057594046185472
AND segment_id = 2; -- this is the rowgroup ID

Run this code in a separate window

In [ ]:
DBCC TRACEON (3604);
GO

DBCC CSINDEX (
    'ParkingTickets', -- db name
    <hobt_id>, -- HoBT ID
    3, -- column ID  (License_type - char(3))
    2, -- segment ID
    1, -- 1 = segment, 2 = dictionary
    0 -- {0,1,2}
);

Cleanup

In [ ]:
DROP INDEX CI_ParkingTickets ON ParkingTickets.dbo.ParkingTickets;